In [1]:
import torchvision.transforms as transforms

F:\laela\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [52]:
from frames_for_specific_scale import Frames_for_specific_scale

In [49]:
from PIL import Image
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

#personal imports
from configuration import *
from create_dataset import Create_dataset
from chunking import Chunking
from detections import *
from frames_for_specific_scale import Frames_for_specific_scale
from image_with_detections import Image_with_detections
import network

In [4]:
def train_neural_network(train_loader, net, optimizer, criterion):
    """
    :param train_loader:
    :param net: NN to train
    :param optimizer:
    :param criterion:
    :return: NN
    """
    start_time = time.time()
    print('Start training')
    for epoch in range(1):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')
    print("time spent:", time.time() - start_time)
    return net

In [5]:
def test_neural_network_original_dataset(test_loader, net):
    """
    :param test_loader:
    :param net: NN already trained
    :return: None
    """
    start_time = time.time()
    print('Start testing')
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    classes = (0, 1,)
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            labels = torch.LongTensor(([1 if label > 1 else label for label in labels]))
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(2):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            labels = torch.LongTensor(([1 if label > 1 else label for label in labels]))
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
            100 * correct / total))
    print('Finished Training')
    print("time spent:", time.time() - start_time)

In [54]:
def test_neural_network_own_dataset(path_to_image, net):
    """
    :param path_to_image: string which indicates where our image is stored
    :param net: NN already trained
    :return: None
    """
    start_time = time.time()
    print('Start testing')

    image = Image.open(path_to_image)
    # Convert into grayscale
    #image = image.convert("L")

    # cut image to different scales
    Chunking.init(image)
    (scaled_images, scaled_positions) = Chunking.get_chunks_of_image_at_different_scales(strides=(5, 5), nb_shrinkages=3, division_factor=2)

    # compute score for each frame
    frames = []
    scaled_factor = 1
    for index in range(len(scaled_images)):
        dataset = Create_dataset(scaled_images[index])
        # get the dataloader. Minibatches of 4
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=4,
                                                 shuffle=True, num_workers=2)
        scores = []
        with torch.no_grad():
            for data in dataloader:
                images, _ = data
                outputs = net(images)  # [(score of class_0, score of class_1), ...]
                outputs = outputs.numpy()
                outputs = outputs.tolist()
                scores.extend(outputs)
        print("just begore to add it to frames")
        frames.append(Frames_for_specific_scale(scaled_factor, dataset, scaled_positions[index], scores))
        scaled_factor = division_factor * (2 ** index)

    # subdetections
    subdetections = capture_subdetections(frames)

    # clustering of Subdetections into clusters_of_subdetections and filtering
    clusters_of_subdetections = get_detections(subdetections, min_samples=2)  
    # min_samples --> number of min detection to confirm the detection
    #print(clusters_of_subdetections)  # In the example, one subdetection is alone in a detection => this detection is discarded.

    # get the best candidates
    winners = get_best_clusters_candidates(subdetections, clusters_of_subdetections)
    for winner in winners:
        #print(winner)
        pass

    # save an image with all subdetections, and then only with the kept subdetections
    image_with_detections = Image_with_detections(image, subdetections)
    original_name_image = path_to_image.split('.')

    # With all subdetections
    image_with_detections.save(original_name_image[0] + "_all_detections.JPG")

    # With only winner subdetections
    image_with_detections.subdetections = winners
    image_with_detections.save(original_name_image[0] + "_winners_detections.JPG")

    print('Finished Training')
    print("time spent:", time.time() - start_time)

In [14]:
#training data set
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
train_imagenet = torchvision.datasets.ImageFolder('start_deep/start_deep/start_deep/train_images/',transform=transform)

#data augmentation
for theta in range(3):
    transform = transforms.Compose(
        [transforms.RandomRotation((90*(theta+1), 90*(theta+1))),
        transforms.ToTensor(),
         transforms.Normalize([0.5], [0.5])])
    train_imagenet += torchvision.datasets.ImageFolder('start_deep/start_deep/start_deep/train_images/',transform=transform)

train_loader = torch.utils.data.DataLoader(train_imagenet, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)

#define net and define optimizer
net = network.Net()
weights = torch.Tensor([1, 3])
criterion = nn.CrossEntropyLoss(weights)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
#training
net = train_neural_network(train_loader, net, optimizer, criterion)

Start training
[1,  2000] loss: 0.386
[1,  4000] loss: 0.236
[1,  6000] loss: 0.193
[1,  8000] loss: 0.172
[1, 10000] loss: 0.145
[1, 12000] loss: 0.114
[1, 14000] loss: 0.099
[1, 16000] loss: 0.091
[1, 18000] loss: 0.076
[1, 20000] loss: 0.080
[1, 22000] loss: 0.070
[1, 24000] loss: 0.066
[1, 26000] loss: 0.063
[1, 28000] loss: 0.056
[1, 30000] loss: 0.063
[1, 32000] loss: 0.055
[1, 34000] loss: 0.048
[1, 36000] loss: 0.049
[1, 38000] loss: 0.048
[1, 40000] loss: 0.036
[1, 42000] loss: 0.049
[1, 44000] loss: 0.042
[1, 46000] loss: 0.038
[1, 48000] loss: 0.041
[1, 50000] loss: 0.039
[1, 52000] loss: 0.032
[1, 54000] loss: 0.033
[1, 56000] loss: 0.030
[1, 58000] loss: 0.033
[1, 60000] loss: 0.031
[1, 62000] loss: 0.031
[1, 64000] loss: 0.035
[1, 66000] loss: 0.031
[1, 68000] loss: 0.032
[1, 70000] loss: 0.026
[1, 72000] loss: 0.029
[1, 74000] loss: 0.027
[1, 76000] loss: 0.032
[1, 78000] loss: 0.030
[1, 80000] loss: 0.028
[1, 82000] loss: 0.028
[1, 84000] loss: 0.026
[1, 86000] loss: 0.

In [16]:
#test with original dataset
test_imagenet = torchvision.datasets.ImageFolder('start_deep/start_deep/start_deep/test_images/',
                                                             transform=transform)
test_loader = torch.utils.data.DataLoader(test_imagenet, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)
test_neural_network_original_dataset(test_loader, net)

Start testing
Accuracy of     0 : 97 %
Accuracy of     1 : 72 %
Accuracy of the network on the 10000 test images: 94 %
Finished Training
time spent: 75.21261429786682


In [ ]:
path_to_image = input("(ex : own_dataset/ex3.JPG). The path to the image to test is :")
test_neural_network_own_dataset(path_to_image, net)

(ex : own_dataset/ex3.JPG). The path to the image to test is :own_dataset/ex3.JPG
Start testing
